In [3]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.applications import EfficientNetB0

# טעינת EfficientNetB0 מראש עם משקולות מאומנות
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # מקפיא את השכבות הבסיסיות כדי להשתמש בתכונות מוכנות

# בניית המודל עם שכבות מותאמות
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # הפחתת פרמטרים ושימור מידע
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),  # למניעת Overfitting
    layers.Dense(196, activation='softmax')  # 196 קטגוריות של רכבים
])

# קומפילציה של המודל
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# הצגת תקציר המודל
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 196)            │       100,548 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,805,991 (18.33 MB)

 Trainable params: 756,420 (2.89 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [5]:
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers, callbacks
from tensorflow.keras.applications import EfficientNetB0

# 🔹 1️⃣ טעינת EfficientNetB0 כמודל בסיס
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# 🔹 2️⃣ שחרור 20 השכבות האחרונות לאימון (Fine-Tuning)
for layer in base_model.layers[-20:]:
    layer.trainable = True

# 🔹 3️⃣ בניית המודל עם שכבות מותאמות
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # הפחתת פרמטרים ושימור מידע
    layers.BatchNormalization(),  # מנרמל את הערכים כדי לייצב את הלמידה
    layers.Dense(256, activation='relu'),  # הורדנו מ-512 ל-256
    layers.Dropout(0.5),  # מניעת Overfitting
    layers.Dense(196, activation='softmax')  # 196 קטגוריות של רכבים
])

# 🔹 4️⃣ קומפילציה של המודל עם Adam + ReduceLROnPlateau
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),  # Learning Rate קטן יותר לאימון יציב
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 🔹 5️⃣ קריאה חכמה לשיפור קצב הלמידה
lr_callback = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# 🔹 6️⃣ הצגת מבנה המודל
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 196)            │        50,372 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,432,999 (16.91 MB)

 Trainable params: 4,388,416 (16.74 MB)

 Non-trainable params: 44,583 (174.16 KB)

In [71]:
def grad_cam(model, img_array):
    """
    מייצר Grad-CAM עבור תמונה נתונה ומודל מאומן.
    """
    # ✅ מבצע חיזוי על תמונה כדי להפעיל את המודל
    _ = model.predict(img_array)

    # מוצא את שכבת ה-Convolution האחרונה
    layer_name = "block7a_project_conv"  # השתמש בשכבה האחרונה שמצאת
    print(f"✅ משתמשים בשכבה {layer_name} ל-Grad-CAM")

    # בונה מודל Grad-CAM שמחזיר את הפלט של השכבה האחרונה + התחזיות
    grad_model = tf.keras.models.Model(
        inputs=[model.input], 
        outputs=[model.get_layer(layer_name).output, model.output]  # ❌ לא להשתמש ב-base_model!
    )

    # מחשב את הגרדיאנטים
    with tf.GradientTape() as tape:
        conv_output, predictions = grad_model(img_array)
        loss = predictions[:, np.argmax(predictions[0])]  # הפסד עבור הקטגוריה החזקה ביותר

    grads = tape.gradient(loss, conv_output)
    pooled_grads = K.mean(grads, axis=(0, 1, 2))

    # חישוב מפה חמה
    heatmap = np.mean(conv_output.numpy() * pooled_grads.numpy(), axis=-1)
    heatmap = np.maximum(heatmap, 0)  # הסרת ערכים שליליים
    heatmap = cv2.resize(heatmap, (224, 224))  # שינוי גודל למידות התמונה

    return heatmap


In [83]:
# מציג את המבנה של המודל
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 196)            │        50,372 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,432,999 (16.91 MB)

 Trainable params: 4,388,416 (16.74 MB)

 Non-trainable params: 44,583 (174.16 KB)